# 감성분석

In [14]:
# 텍스트의 긍정 부정 등 감정을 구분하는 것
# 네이버 댓글 

In [134]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [135]:
train_df = pd.read_csv('./data/ratings_train.txt', sep= '\t')
test_df = pd.read_csv('./data/ratings_test.txt', sep= '\t')

In [136]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [137]:
train_df.shape

(150000, 3)

In [138]:
train_df.loc[100,'document']

'신카이 마코토의 작화와,미유와 하나카나가 연기를 잘해줘서 더대박이였다.'

In [139]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [140]:
test_df.shape

(50000, 3)

In [141]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


## 전처리하기

In [142]:
# Nan값 작으므로 없애주기
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [143]:
from konlpy.tag import Komoran
import re

In [144]:
# 형태소 분석기
kor = Komoran()

In [145]:
def text_clean(doc):
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ ]', '',doc) # ^:not -> 한글,스페이스 아닌것을 제거해라
    return text

In [146]:
train_df['document1'] = train_df.document.apply(text_clean)
test_df['document1'] = test_df.document.apply(text_clean)

In [147]:
train_df

,id,document,label,document1
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙 진짜 짜증나네요 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너무재밓었다그래서보는것을추천한다
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기구먼 솔직히 재미는 없다평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...
...,...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0,인간이 문제지 소는 뭔죄인가
149996,8549745,평점이 너무 낮아서...,1,평점이 너무 낮아서
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1,청춘 영화의 최고봉방황과 우울했던 날들의 자화상


In [148]:
# stopword : 한글은 직접 만들어야함
# 불용어 : 텍스트를 분석할 때 의미를 가지지 않는 형태소

stw = pd.read_csv('./data/stopwords-ko.txt',header=None)
stopwords = stw.iloc[:,0].tolist()
stopwords

['가',
 '가까스로',
 '가령',
 '각',
 '각각',
 '각자',
 '각종',
 '갖고말하자면',
 '같다',
 '같이',
 '개의치않고',
 '거니와',
 '거바',
 '거의',
 '것',
 '것과 같이',
 '것들',
 '게다가',
 '게우다',
 '겨우',
 '견지에서',
 '결과에 이르다',
 '결국',
 '결론을 낼 수 있다',
 '겸사겸사',
 '고려하면',
 '고로',
 '곧',
 '공동으로',
 '과',
 '과연',
 '관계가 있다',
 '관계없이',
 '관련이 있다',
 '관하여',
 '관한',
 '관해서는',
 '구',
 '구체적으로',
 '구토하다',
 '그',
 '그들',
 '그때',
 '그래',
 '그래도',
 '그래서',
 '그러나',
 '그러니',
 '그러니까',
 '그러면',
 '그러므로',
 '그러한즉',
 '그런 까닭에',
 '그런데',
 '그런즉',
 '그럼',
 '그럼에도 불구하고',
 '그렇게 함으로써',
 '그렇지',
 '그렇지 않다면',
 '그렇지 않으면',
 '그렇지만',
 '그렇지않으면',
 '그리고',
 '그리하여',
 '그만이다',
 '그에 따르는',
 '그위에',
 '그저',
 '그중에서',
 '그치지 않다',
 '근거로',
 '근거하여',
 '기대여',
 '기점으로',
 '기준으로',
 '기타',
 '까닭으로',
 '까악',
 '까지',
 '까지 미치다',
 '까지도',
 '꽈당',
 '끙끙',
 '끼익',
 '나',
 '나머지는',
 '남들',
 '남짓',
 '너',
 '너희',
 '너희들',
 '네',
 '넷',
 '년',
 '논하지 않다',
 '놀라다',
 '누가 알겠는가',
 '누구',
 '다른',
 '다른 방면으로',
 '다만',
 '다섯',
 '다소',
 '다수',
 '다시 말하자면',
 '다시말하면',
 '다음',
 '다음에',
 '다음으로',
 '단지',
 '답다',
 '당신',
 '당장',
 '대로 하다',
 '대하면',
 '대하여',
 '대해 말하자면',
 '대해서',
 '댕그',


In [149]:
def remove_stopwords(doc):
    text = [w for w in doc.split() if w not in stopwords]
    return ' '.join(text) # 리스트 -> 문자열 하나로 출력

In [150]:
train_df['document2'] = train_df.document1.apply(remove_stopwords)
test_df['document2'] = test_df.document1.apply(remove_stopwords)

In [151]:
train_df.head()

,id,document,label,document1,document2
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙 진짜 짜증나네요 목소리,더빙 진짜 짜증나네요 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너무재밓었다그래서보는것을추천한다,너무재밓었다그래서보는것을추천한다
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기구먼 솔직히 재미는 없다평점 조정,교도소 이야기구먼 솔직히 재미는 없다평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...


In [152]:
test_df.head()

,id,document,label,document1,document2
0,6270596,굳 ㅋ,1,굳 ㅋ,굳 ㅋ
1,9274899,GDNTOPCLASSINTHECLUB,0,,
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,뭐야 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,지루하지는 않은데 완전 막장임 돈주고 보기에는,지루하지는 않은데 완전 막장임 돈주고 보기에는
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,만 아니었어도 별 개 줬을텐데 나와서 심기를 불편하게 하죠


In [153]:
# 처리 안된값들
train_df[train_df['document2']== '']

,id,document,label,document1,document2
151,7348295,아,0,아,
404,4221289,What is this movie for?,0,,
412,9509970,Yesterday when i was young,1,,
470,10147571,once upon a dream,1,,
584,7117896,1,0,,
...,...,...,...,...,...
149630,3508604,"So, what ?",0,,
149718,7690797,.,1,,
149773,9233162,Real Trash Movie,0,,
149862,7175749,.,0,,


In [154]:
# 처리 안된값들 없애주기
train_df = train_df[train_df['document2'] != '']
test_df = test_df[test_df['document2'] != '']

## 토큰나이징: 명사만 추출

In [155]:
train_docs = []

for d in train_df['document2']:
    tokens = kor.nouns(d)
    train_docs.append(tokens)

In [156]:
train_docs

[['더빙', '짜증', '목소리'],
 ['흠', '포스터', '초딩', '영화', '줄', '오버', '연기'],
 [],
 ['교도소', '이야기', '재미', '평점', '조정'],
 ['익살', '연기', '영화', '스파이더맨', '커스틴 던스트'],
 ['걸음마', '세', '초등학교', '학년'],
 ['원작', '긴장감'],
 ['반개',
  '욕',
  '이응경',
  '길용우',
  '연기',
  '생활',
  '이',
  '년',
  '인지',
  '정말',
  '발로',
  '납치',
  '감금',
  '만',
  '반복',
  '반복',
  '이',
  '드라마',
  '가족',
  '연기',
  '사람'],
 ['액션', '재미', '안', '영화'],
 ['평점', '건', '데', '만', '헐리우드', '식'],
 [],
 ['볼', '때', '눈물', '년대', '향수', '자극', '허진호', '감성', '절제', '멜로', '달인', '이다'],
 ['횡단보도', '때', '뻔', '이범수', '연기'],
 ['신문', '기사', '로만', '사람', '것'],
 ['취향', '존중', '극장', '영화', '중', '노', '노', '감동', '스토리', '어거지', '감동', '어거지'],
 ['긴장'],
 ['사람', '바스코', '락스', '코', '까', '고', '바비', '아이돌', '안달', '난', '것'],
 ['굿바이 레닌', '표절', '것', '이해', '뒤'],
 ['이건', '깨알', '캐스팅', '내용', '구성', '버', '러', '진', '깨알', '드'],
 ['약탈', '자', '변명', '놈', '절대'],
 ['나름', '뜻', '듯', '학생', '선생', '영화', '절대'],
 ['건'],
 ['음식', '영화', '만찬', '넘', '만찬', '이야기', '음식', '재미', '이건', '음식', '핀란드', '풍경'],
 ['절대', '영화', '수작', '거', '말씀']

In [157]:
for i in range(len(train_docs)):
    train_docs[i] = ' '.join(train_docs[i])

In [158]:
train_docs

['더빙 짜증 목소리',
 '흠 포스터 초딩 영화 줄 오버 연기',
 '',
 '교도소 이야기 재미 평점 조정',
 '익살 연기 영화 스파이더맨 커스틴 던스트',
 '걸음마 세 초등학교 학년',
 '원작 긴장감',
 '반개 욕 이응경 길용우 연기 생활 이 년 인지 정말 발로 납치 감금 만 반복 반복 이 드라마 가족 연기 사람',
 '액션 재미 안 영화',
 '평점 건 데 만 헐리우드 식',
 '',
 '볼 때 눈물 년대 향수 자극 허진호 감성 절제 멜로 달인 이다',
 '횡단보도 때 뻔 이범수 연기',
 '신문 기사 로만 사람 것',
 '취향 존중 극장 영화 중 노 노 감동 스토리 어거지 감동 어거지',
 '긴장',
 '사람 바스코 락스 코 까 고 바비 아이돌 안달 난 것',
 '굿바이 레닌 표절 것 이해 뒤',
 '이건 깨알 캐스팅 내용 구성 버 러 진 깨알 드',
 '약탈 자 변명 놈 절대',
 '나름 뜻 듯 학생 선생 영화 절대',
 '건',
 '음식 영화 만찬 넘 만찬 이야기 음식 재미 이건 음식 핀란드 풍경',
 '절대 영화 수작 거 말씀',
 '주제 중반',
 '꺼 납득 수 거 꺼 걸',
 '고추',
 '카밀라 벨 발 연기',
 '',
 '센스 연출력 캐스팅 년대 향수 점',
 '엄 포스 위력 번 적 남 꽃 검사 연기 완전 명품 드라마',
 '졸 쓰레기 아시 간',
 '별점 낮 은고',
 '라도 기대 죄인 죄인',
 '드라마 내 인생 최고',
 '패션 열정 안나 윈 투어',
 '키이라 나이틀리 연기 건 대체 정신 장애 틱 장애',
 '허허 원 작가 정신 유령',
 '포스터 관객 명',
 '영화 평가',
 '매력 영화',
 '알바 내용 거도 거도 완전 별 넘 완전',
 '서리 한',
 '맘 방법',
 '윤제문 배우 발견 일탈 미소 음악 점 평점 점',
 '평점 속지 마시 길 시간 낭비 돈 낭비',
 '리얼리티 공감 간다 이민기 캐릭터 정신의학 분노 조절 장애 초기 증상 일거 사람 욕 물건 파손 오바 극 초반 이민기 정신 상태 공감 불가',
 '마이너스

In [164]:
test_docs= []

for d in test_df['document2']:
    tokens = kor.nouns(d)
    test_docs.append(tokens)

In [165]:
for i in range(len(test_docs)):
    test_docs[i] = ' '.join(test_docs[i])

In [166]:
train_df['train_docs']= train_docs
test_df['test_docs'] = test_docs

C:\Users\bitcamp\AppData\Local\Temp/ipykernel_16056/914006210.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['train_docs']= train_docs


## 벡터화

In [167]:
from sklearn.feature_extraction.text import CountVectorizer

### 1. CountVectorize

In [178]:
vec = CountVectorizer(min_df= 3, max_df= 0.9, max_features= 2000) # 최소등장횟수:3 , 최대등장비율: 0.9
x_train = vec.fit_transform(train_docs)
x_train

<148611x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 440398 stored elements in Compressed Sparse Row format>

In [169]:
x_train.toarray().shape

(148611, 2000)

In [170]:
x_train_vec = x_train.toarray()

In [171]:
vec = CountVectorizer(min_df=3, max_df=0.9, max_features=2000)
x_test = vec.fit_transform(test_docs)
x_test

<49520x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 148026 stored elements in Compressed Sparse Row format>

In [172]:
x_test.toarray().shape

(49520, 2000)

In [173]:
x_test_vec = x_test.toarray()

In [174]:
y_train = train_df.label
y_test = test_df.label

### Countvectorize 머신러닝

In [175]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression()
log.fit(x_train_vec, y_train)

C:\Users\bitcamp\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [176]:
preds = log.predict(x_test_vec)
preds

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

In [177]:
from sklearn.metrics import classification_report
print(classification_report(y_test,preds))

              precision    recall  f1-score   support

           0       0.50      0.73      0.59     24614
           1       0.51      0.28      0.36     24906

    accuracy                           0.50     49520
   macro avg       0.51      0.51      0.48     49520
weighted avg       0.51      0.50      0.48     49520



### 2. TfidVectorize

In [179]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [190]:
vec = TfidfVectorizer(min_df=0.1, max_df= 0.9, ngram_range=(1,2), max_features=2000)
x_train = vec.fit_transform(train_df,train_docs)

In [191]:
x_train_vec = x_train.toarray()

In [192]:
y_train = train_df['label']

In [193]:
x_test_vec = x_test.toarray()

In [194]:
y_test = test_df.label

### Tfid 머신러닝

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(x_train_vec, y_train)

In [ ]:
preds = rfc.predict(x_test_vec)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))